Extracting sentences from the corpora
---------------------

In this notebook we will try to extract the different sentences in the corpora. Note that it can be very difficult because a sentence in french can be related to two or many sentences in wolof and some sentences are moved from their right places. It will require to take in account some options in order to manually select the related sentences between the French corpus and the Wolof corpus:

- We will look at the semantic of a french sentence to select the different related Wolof sentences. 
- If the french sentence is not related to any wolof sentence we will skip its selection.
- If a french sentence is not totally describe by a wolof sentence we will search the rest of the semantic in the next wolof sentences until completing it. And vice versa for a wolof sentence. 
- If a sentence contains an expression beginning with "«" or "(" and not being closed then the found sentence is not a true one and we must concatenated it with the next sentence. The same is done when the expression is closed but doesn't have a beginning but at this time we concatenated it with the previous sentence.  
- It will require that the translation to be as correct as possible.
- We will use the extracted paragraphs in order to find the different related sentences.
- We will process one by one the paragraphs to not make errors.
- We will save the related sentences after each paragraph processing.

Let us load the corpora.

In [2]:
import pandas as pd

!pip install -q -e wolof-translate

from typing import *

corpora_v2 = pd.read_csv("data/extractions/new_data/corpora_v2.csv", encoding='utf-16') 

corpora = corpora_v2.copy() # copy


[notice] A new release of pip available: 22.3 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


We will need to create a class to extract the sentences.

In [3]:
from nlp_project import *
import pickle

class LengthError(Exception):
    
    def __init__(self, *args: object) -> None:
        super().__init__(*args)
        
class CommandError(Exception):
    
    def __init__(self, *args: object) -> None:
        super().__init__(*args)

class ExtractRelatedSentences:
    
    nb_process = 1
    
    indices = {}
    
    def __init__(self, corpora: pd.DataFrame, corpus_1: str = "french_corpus", corpus_2: str = "wolof_corpus"):
        
        self.corpora = corpora
        
        self.corpus_1 = corpus_1
        
        self.corpus_2 = corpus_2
        
        self.length = corpora.shape[0]
        
        self.sentences = {}
        
        self.passed = {}
        
    @classmethod
    def reload(cls, number: int = 1):
        
        cls.nb_process = number
        
    @classmethod
    def store_indices(cls, nb_paragraph, i: int, j: int):
        
        cls.indices[nb_paragraph] = {'i': i, 'j': j}
        
    def increment(self, number: int = 1):
        
        ExtractRelatedSentences.nb_process += number
        
        if ExtractRelatedSentences.nb_process > self.length:
            
            raise ValueError("The last paragraph is reached!")
        
    def decrement(self, number: int = 1):
        
        ExtractRelatedSentences.nb_process -= number
        
        if ExtractRelatedSentences.nb_process < 1:
            
            ExtractRelatedSentences.nb_process = 1
    
    def add_sentences(self, nb_paragraph: int, sentences: dict):
        
        self.sentences[nb_paragraph] = sentences
        
        if len(sentences['1']) != len(sentences['2']):
            
            raise LengthError("The number of sentences in the two corpora must be equal!")
    
    def add_passed(self, nb_paragraph: int, sentences: dict):
        
        self.sentences[nb_paragraph] = sentences
    
    def clear_sentences(self, nb_paragraph: int):
        
        sentences = self.sentences[nb_paragraph]
        
        clear = input(f"Are you sure you want to remove the following sentences!\
            \n\nOn {self.corpus_1}:\n{sentences['1']}\n\nOn {self.corpus_2}:\n{sentences['2']}\nYes(y), No(n) :")
        
        if clear == 'y':
            
            del sentences[nb_paragraph]
            
            print(f"Sentences at {nb_paragraph} was cleared!")
        
        elif clear == 'n':
            
            print(f"Process aborted!")
        
        else:
            
            raise CommandError(f"You cannot take the command {clear}!")     
    
    def get_sentences(self, nb_paragraph: int):
        
        return self.sentences[nb_paragraph] 
        
    def unify(self, sentences: list, sentence: str, unification_marks: list):
        
        for mark in unification_marks:
            
            begin_mark = False
            
            end_mark = False
            
            for letter in sentences[-1]:
                
                if letter == mark[1]:
                    
                    begin_mark = False
                
                elif letter == mark[0]:
                    
                    begin_mark = True
            
            for letter in sentence:
                
                if letter == mark[1]:
                    
                    end_mark = True

                    break
                
                else:
                    
                    break
            
            if sentence != "" and sentence[0].islower():
                
                return True, " "
            
            if end_mark or begin_mark:
                
                space = " " if mark[2] else ""
                
                return True, space
                    
        return False, " "
    
    def split(self, paragraph: str, ending_marks: list, unification_marks: list):
        
        prob_sentences = paragraph.strip()
        
        for mark in ending_marks:
            
            if isinstance(prob_sentences, list):
                
                new_sentences = prob_sentences.copy()
                
                counter = 0
                
                for s in prob_sentences:
                    
                    if mark in s:
                        
                        splits = new_sentences[counter].split(mark)
                        
                        sentences = [sentence.strip() + mark for sentence in splits[:-1]] + splits[-1:]
                        
                        new_sentences = new_sentences[:counter] + sentences + new_sentences[counter + 1:] 
                        
                        counter += len(sentences) - 1
                        
                    else:
                        
                        counter += 1
                
                prob_sentences = new_sentences
                
            else:
                
                if mark in prob_sentences:
                    
                    splits = prob_sentences.split(mark)
                    
                    prob_sentences = [sentence.strip() + mark for sentence in splits[:-1]] + splits[-1:]
                
            new_sentences = []
                
            counter = 0
                
            for s in prob_sentences:
                
                unify, space = False, "" 
                
                if counter != 0:
                    
                    unify, space = self.unify(new_sentences, s, unification_marks)
                
                if s != '': 
                    
                    if not unify:
                        
                        new_sentences.append(s)
                        
                        counter += 1
                    
                    else:
                        
                        new_sentences[-1] = new_sentences[-1] + space + s
            
            prob_sentences = new_sentences
        
        return prob_sentences
    
    def __save(self, storage: str = "data/extractions/new_data/sent_extraction.txt"):
        
        with open(storage, "wb") as f:
            
            checkpoints = {
                'indices': ExtractRelatedSentences.indices,
                'nb_process': ExtractRelatedSentences.nb_process,
                'sentences': self.sentences,
                'passed': self.passed
            }
            
            pickler = pickle.Pickler(f)
            
            pickler.dump(checkpoints)
    
    def load(self, storage: str = "data/extractions/new_data/sent_extraction.txt"):
        
        with open(storage, "rb") as f:
            
            depickler = pickle.Unpickler(f)
            
            checkpoints = depickler.load()
            
            ExtractRelatedSentences.indices = checkpoints['indices']
            
            ExtractRelatedSentences.nb_process = checkpoints['nb_process']
            
            self.sentences = checkpoints['sentences']
            
            self.passed = checkpoints['passed']
    
    def preprocess(self, 
                   number: Union[int, None] = None,
                   ending_marks: list = ['.', ' ?', ' !'], 
                   unification_marks: List[Tuple] = [("«", "»", True), ("(", ")", True), ('“', '”', True)],
                   cr: str = "r", cm1: str = 'f', cm2: str = 'j', cm3: str = 'l',
                   cmp1: str = 'y', cmp2: str = 'i', cmp3: str = 'p', q: str = 'q', 
                   start_at_last_indices: bool = False,
                   i: int = 0, j: int = 0,
                   auto_save: bool = True,
                   storage: str = "data/extractions/new_data/sent_extraction.txt",
                   ):
        
        line = number if not number is None else self.nb_process
        
        process_again = ''
        
        try:
            self.load(storage=storage)
        except:
            pass
        
        if line in set(self.sentences):
            
            process_again = input(f"You have already process the paragraph at line {line}.\nDo you want to modify from the processed sentences ? Yes(y), No(n):")

        print(f"Preprocessing of paragraph at line {line}")
        
        if process_again == 'n' or process_again == '':
            
            paragraph1 = str(self.corpora.loc[line-1, self.corpus_1])
            
            paragraph2 = str(self.corpora.loc[line-1, self.corpus_2])
            
            # let us separate the paragraphs by ending marks
            
            prob_sentences1 = self.split(paragraph1, ending_marks, unification_marks)
            
            prob_sentences2 = self.split(paragraph2, ending_marks, unification_marks)
        
        elif process_again == 'y':
            
            prob_sentences1 = self.sentences[line]['1']
            
            prob_sentences2 = self.sentences[line]['2']
        
        else:
            
            raise CommandError(f"You cannot take the command {process_again}!")
        
        print("\n-----------\n-----------\n")
        
        print("Do you want to process the following sentences:\n")
        
        print(f"On {self.corpus_1}: ")
        
        [print(f"{i}: {sentence}") for i, sentence in enumerate(prob_sentences1)]
        
        print(f"\nOn {self.corpus_2}: ")
        
        [print(f"{i}: {sentence}") for i, sentence in enumerate(prob_sentences2)]
        
        print("\n-----------")
        
        process = input("Yes(y), Accept all (a) or No(n): ")
        
        cm = ''
        
        if process == 'y':
        
            sentences = {'1': [], '2': []}
            
            passed = {'1': [], '2': []}
            
            last_sentences = {'1': '', '2': ''}
            
            if start_at_last_indices and line in set(self.indices):
                
                i = self.indices[line]['i']
                
                j = self.indices[line]['j']
            
            while i < len(prob_sentences1) and j < len(prob_sentences2):
                
                self.store_indices(line, i, j)
                
                sentence1 = sentence1_ = prob_sentences1[i]
                
                sentence2 = sentence2_ = prob_sentences2[j]
                
                if last_sentences['1'] != '':
                        
                    sentence1_ = last_sentences['1'].strip() + " " + sentence1
                
                if last_sentences['2'] != '':
                    
                    sentence2_ = last_sentences['2'] + " " + sentence2
                
                print(f"\nThe current sentences are:\n{self.corpus_1} (index = {i}) : {sentence1_}\n{self.corpus_2} (index = {j}) : {sentence2_}")
                
                cm = input(f"Are they related together ?\n(index = {i}) : {sentence1_}\n{self.corpus_2} (index = {j}) : {sentence2_}\nQuit ({q}), Related ({cr}), Sentence 1 is uncompleted ({cm1}), Sentence 2 is uncompleted ({cm2}), The two sentences are uncompleted ({cm3}),\n Pass sentence 1 ({cmp1}), Pass sentence 2 ({cmp2}), Pass the two sentences ({cmp3}) :")
                
                if cm == cr:
            
                    # clear the last sentences
                    last_sentences = {'1': '', '2': ''}
                    
                    # add the sentences to the list of related sentences
                    sentences['1'].append(sentence1_.strip())
                    
                    sentences['2'].append(sentence2_.strip())
                    
                    # Pass to the next sentences
                    i += 1
                    
                    j += 1
                    
                elif cm == cm1:
                    
                    # The first sentence is added to the last sentence 1
                    last_sentences['1'] += " " + sentence1
                    
                    # Pass to the next sentence at corpus 1
                    i += 1
                
                elif cm == cm2:
                    
                    # The second sentence is added to the last sentence 2
                    last_sentences['2'] += " " + sentence2
                    
                    # Pass to the next sentence at corpus 2
                    j += 1
                
                elif cm == cm3:
                    
                    # The two sentences are added to the last sentences
                    last_sentences['1'] += " " + sentence1
                    
                    last_sentences['2'] += " " + sentence2
                    
                    # Pass to the next sentences
                    i += 1
                    
                    j += 1
                
                elif cm == cmp1:
                    
                    # Clear the last sentence at corpus 1
                    last_sentences['1'] = ''
                    
                    # Add the sentence 1 to the passed sentences at corpus 1
                    passed['1'].append(sentence1_)
                    
                    # Pass to the next sentence at corpus 1
                    i += 1
                
                elif cm == cmp2:
                    
                    # Clear the last sentence at corpus 2
                    last_sentences['2'] = ''
                    
                    # Add the sentence 2 to the passed sentences at corpus 2
                    passed['2'].append(sentence2_)
                    
                    # Pass to the next sentence at corpus 2
                    j += 1
                
                elif cm == cmp3:
                    
                    # Clear the last sentences
                    last_sentences = {'1': '', '2': ''}
                    
                    # Add the two sentences to the passed sentences 
                    passed['1'].append(sentence1_)
                    
                    passed['2'].append(sentence2_)
                    
                    # Pass to the next sentences 
                    i += 1
                    
                    j += 1
                
                elif cm == q:
                    
                    break
                    
                else:
                    
                    print(f"You cannot take the command {cm} ! Please retry again !")

                    print("\n-----------\n")
                    
                    continue
                
                print("\nYou have stored the following sentences for the moment :\n")
                
                print(f"On {self.corpus_1} (length = {len(sentences['1'])}): ")
        
                print("\n".join(sentences['1']))
                
                print(f"\nOn {self.corpus_2} (length = {len(sentences['2'])}): ")
                
                print("\n".join(sentences['2']))
                
                print("\n-----------\n")
                
                print("You have passed the following sentences :\n")
                
                print(f"On {self.corpus_1} (length = {len(passed['1'])}): ")
        
                print("\n".join(passed['1']))
                
                print(f"\nOn {self.corpus_2} (length = {len(passed['2'])}): ")
                
                print("\n".join(passed['2']))
                
                print("\n------------------------------------")
                
                # add the sentences
                self.add_sentences(line, sentences)
                
                # add the passed sentences
                self.add_passed(line, passed)
                
                if auto_save:
                    # save the checkpoints
                    self.__save(storage)
            
            print("\nFinished!")
            
            # incrementing the number of processes
            self.increment()
            
            # add the passed sentences
            self.add_passed(line, passed)
            
            # add the sentences
            self.add_sentences(line, sentences)
            
            if auto_save:
                    # save the checkpoints
                    self.__save(storage)
        
        elif process == 'a':
            
            sentences = {'1': prob_sentences1, '2': prob_sentences2}
            
            print("\nFinished!")
            
            # incrementing the number of processes
            self.increment()
            
            # add the sentences
            self.add_sentences(line, sentences)
            
            if auto_save:
                    # save the checkpoints
                    self.save(storage)
        
        elif process == 'n' or cm == 'q':
            
            print(f"Process aborted!")
        
        else:
            
            raise CommandError(f"You cannot take the command {process}!")
            
            

Let us begin the sentence extraction.

In [4]:
sent_extraction = ExtractRelatedSentences(corpora)

In [5]:
sent_extraction.reload()

In [74]:
sent_extraction.preprocess()

Preprocessing of paragraph at line 1

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Tout être humain est le résultat d’un père et une mère.
1: On peut ne pas les reconnaître, ne pas les aimer, on peut douter d’eux.
2: Mais ils sont là, avec leur visage, leurs attitudes, leurs manières et leurs manies, leurs illusions, leurs espoirs, la forme de leurs mains et de leurs doigts de pied, la couleur de leurs yeux et de leurs cheveux, leur façon de parler, leurs pensées, probablement l’âge de leur mort, tout cela est passé en nous.

On wolof_corpus: 
0: Doomu-aadama bu, ne ci ndey ak baay nga jóge.
1: Mënunu leen a baň a gërëm ak a bëgg, doonte sax mën nanoo am xel ňaar ci ňoom.
2: Waaye ňu ngi fi, ak seen xar-kanam, seen taxawaay, seen defin ak seen jikko, seeni njuumte, seeni yaakaar, seen melokaanu loxook baaraami tànk, seen meloy bët ak karaw, seen waxin, seeni xalaat, amaana sax at ma ňuy nar a génne àddina.
3: Loolu lépp, day àgg fu sor

In [81]:
sent_extraction.preprocess(2)

Preprocessing of paragraph at line 2

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: J’ai longtemps rêvé que ma mère était noire.
1: Je m’étais inventé une histoire, un passé, pour fuir la réalité à mon retour d’Afrique, dans ce pays, dans cette ville où je ne connaissais personne, où j’étais devenu un étranger.
2: Puis j’ai découvert, lorsque mon père, à l’âge de la retraite, est revenu vivre avec nous en France, que c’était lui l’Africain.
3: Cela a été difficile à admettre.
4: Il m’a fallu retourner en arrière, recommencer, essayer de comprendre.
5: En souvenir de cela, j’ai écrit ce petit livre.

On wolof_corpus: 
0: Bi ma delloo dëkk ba ma juddoo, dama faa meloon ni gan.
1: Du kenn ku ma fa xam, safatul dara ci man.
2: Li nu jóge Afrig jur ci man tiis wu réy.
3: Su ma yaboo sax ni mënuma woon a nangu ni maak samay way-jur dëkkëtuñu Niseryaa.
4: Ca laa tàmbalee gént ni sama yaay nit ku ñuul la, di sàkkal it sama bopp cosaan lu bees.


In [84]:
sent_extraction.preprocess(3)

Preprocessing of paragraph at line 3

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: De ce visage que j’ai reçu à ma naissance, j’ai des choses à dire.
1: D’abord, qu’il m’a fallu l’accepter.
2: Affirmer que je ne l’aimais pas serait lui donner une importance qu’il n’avait pas quand j’étais enfant.
3: Je ne le haïssais pas, je l’ignorais, je l’évitais.
4: Je ne le regardais pas dans les miroirs.
5: Pendant des années, je crois que je ne l’ai jamais vu.
6: Sur les photos, je détournais les yeux, comme si quelqu’un d’autre s’était substitué à moi.

On wolof_corpus: 
0: Kanam gii ma judduwaale, am na lu bari lu ma ci mën a wax.
1: Li ci jiitu moo di ne dama dem ba jàppe ko nattu bu ma war a nangu.
2: Su ma nee soppuma ko woon, day mel ni dama koy jox maanaa ju mu amul woon ba may gone.
3: Duma ne it baňoon naa ko.
4: Dama ko daan fàtte, di ko moytu ba daawuma sax seetu.
5: Su ma ragalul woon fen, dinaa ni toog naa ay at teguma bët sama xar-

In [85]:
sent_extraction.preprocess(4)

Preprocessing of paragraph at line 4

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: À l’âge de huit ans à peu près, j’ai vécu en Afrique de l’Ouest, au Nigeria, dans une région assez isolée où, hormis mon père et ma mère, il n’y avait pas d’Européens, et où l’humanité, pour l’enfant que j’étais, se composait uniquement d’Ibos et de Yoroubas.
1: Dans la case que nous habitions (le mot case a quelque chose de colonial qui peut aujourd’hui choquer, mais qui décrit bien le logement de fonction que le gouvernement anglais avait prévu pour les médecins militaires, une dalle de ciment pour le sol, quatre murs de parpaing sans crépi, un toit de tôle ondulée recouvert de feuilles, aucune décoration, des hamacs accrochés aux murs pour servir de lits et, seule concession au luxe, une douche reliée par des tubes de fer à un réservoir sur le toit que chauffait le soleil), dans cette case, donc, il n’y avait pas de miroirs, pas de tableaux, rien qui 

In [89]:
sent_extraction.preprocess(5)


Preprocessing of paragraph at line 5

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: De ce temps, pour ainsi dire consécutivement, date l’apparition des corps.
1: Mon corps, le corps de ma mère, le corps de mon frère, le corps des jeunes garçons du voisinage avec qui je jouais, le corps des femmes africaines dans les chemins, autour de la maison, ou bien au marché, près de la rivière.
2: Leur stature, leurs seins lourds, la peau luisante de leur dos.
3: Le sexe des garçons, leur gland rose circoncis.
4: Des visages sans doute, mais comme des masques de cuir, endurcis, couturés de cicatrices, de marques rituelles.
5: Les ventres saillants, le bouton du nombril pareil à un galet cousu sous la peau.
6: L’odeur des corps aussi, le toucher, la peau non pas rude mais chaude et légère, hérissée de milliers de poils.
7: J’ai cette impression de la grande proximité, du nombre des corps autour de moi, quelque chose que je n’avais pas connu auparav

In [93]:
sent_extraction.preprocess(6)


Preprocessing of paragraph at line 6

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: En Afrique, l’impudeur des corps était magnifique.
1: Elle donnait du champ, de la profondeur, elle multipliait les sensations, elle tendait un réseau humain autour de moi.
2: Elle s’harmonisait avec le pays ibo, avec le tracé de la rivière Aiya, avec les cases du village, leurs toits couleur fauve, leurs murs couleur de terre.
3: Elle brillait dans ces noms qui entraient en moi et qui signifiaient beaucoup plus que des noms de lieux : Ogoja, Abakaliki, Enugu, Obudu, Baterik, Ogrude, Obubra.
4: Elle imprégnait la muraille de la forêt pluvieuse qui nous enserrait de toutes parts.

On wolof_corpus: 
0: Ca Afrig, ňàkk kersag yaram yi lu yéemoon na ma lool.
1: Daan na yaatal, di xóotal, tey yokk yëg-yëg.
2: Daan na maasaloo yit ak réewu Ibo yi, seeni néegi-ñax yaak dexug Ayaa, ba ca sax jànq yaak miir ya amoon melo suuf.
3: May yëgaat ni muy nes-nesilee ay t

In [95]:
sent_extraction.preprocess(7)

Preprocessing of paragraph at line 7

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Quand on est enfant, on n’use pas de mots (et les mots ne sont pas usés).
1: Je suis en ce temps-là très loin des adjectifs, des substantifs.
2: Je ne peux pas dire ni même penser : admirable, immense, puissance.
3: Mais je suis capable de le ressentir.
4: À quel point les arbres aux troncs rectilignes s’élancent vers la voûte nocturne fermée au-dessus de moi, enfermant comme dans un tunnel la brèche sanglante de la route de latérite qui va d’Ogoja vers Obudu, à quel point dans les clairières des villages je ressens les corps nus, brillants de sueur, les silhouettes larges des femmes, les enfants accrochés à leur hanche, tout cela qui forme un ensemble cohérent, dénué de mensonge.

On wolof_corpus: 
0: Gone, du jëfandikooy baat.
1: Rax-ci-dolli, baat bu nekk day wéy di bees ci moom ba fàww.
2: Amu ci woon benn bu ma xam naka la ko nëwu biy tudde.
3: Kàdd

In [96]:
sent_extraction.preprocess(8)


Preprocessing of paragraph at line 8

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: L’entrée dans Obudu, je m’en souviens bien : la route sort de l’ombre de la forêt et entre tout droit dans le village, en plein soleil.
1: Mon père a arrêté son auto, avec ma mère il doit parler aux officiels.
2: Je suis seul au milieu de la foule, je n’ai pas peur.
3: Les mains me touchent, passent sur mes bras, sur mes cheveux autour du bord de mon chapeau.

On wolof_corpus: 
0: Sooy dugg Óbudu, dangay won àll bi ginnaaw.
1: Noonu lanu def bés ban fa njëkkee teg tànk, nestuut nu séen jant bi, daldi xam ni yeg nanu.
2: Ci lan wàccee woto bi, samay way-jur dem nuyoonteek kilifay dëkk bi.
3: Ma nekk gone gu tuuteek sama weexaay : mënuma réer ci mbooloo mi, fu nit tollu di ma séen.
4: Wànte taxumaa tiit benn yoon.
5: Maa ngi yëg ni ma nit ñiy làmbatoo, ni ñuy xëcce sama loxo, di laal it samay karaw.

-----------

The current sentences are:
french_corpus (i

In [106]:
sent_extraction.preprocess(9)

Preprocessing of paragraph at line 9

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Parmi tous ceux qui se pressent autour de moi, il y a une vieille femme, enfin je ne sais pas qu’elle est vieille.
1: Je suppose que c’est d’abord son âge que je remarque, parce qu’elle diffère des enfants nus et des hommes et des femmes habillés plus ou moins à l’occidentale que je vois à Ogoja.
2: Quand ma mère revient (peut-être vaguement inquiète de ce rassemblement), je lui montre cette femme : « Qu’est-ce qu’elle a ? Est-ce qu’elle est malade ? » Je me souviens de cette question que j’ai posée à ma mère.
3: Le corps nu de cette femme, fait de plis, de rides, sa peau comme une outre dégonflée, ses seins allongés et flasques, pendant sur son ventre, sa peau craquelée, ternie, un peu grise, tout cela me semble étrange, et  en même temps vrai.
4: Comment aurais-je pu imaginer que cette femme était ma grand-mère ? Et je ressentais non pas de l’horreur n

In [112]:
sent_extraction.preprocess(10)


Preprocessing of paragraph at line 10

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: L’Afrique, c’était le corps plutôt que le visage.
1: C’était la violence des sensations, la violence des appétits, la violence des saisons.
2: Le premier souvenir que j’ai de ce continent, c’est mon corps couvert d’une éruption de petites ampoules causées par l’extrême chaleur, une affection bénigne dont souffrent les Blancs à leur entrée dans la zone équatoriale, sous le nom comique de « bourbouille » – en anglais prickly heat.
3: Je suis dans la cabine du bateau qui longe lentement la côte, au large de Conakry, Freetown, Monrovia, nu sur la couchette, hublot ouvert sur l’air humide, le corps saupoudré de talc, avec l’impression d’être dans un sarcophage invisible, ou d’avoir été pris comme un poisson dans la nasse, enfariné avant d’aller à la friture.
4: L’Afrique qui déjà m’ôtait mon visage me rendait un corps, douloureux, enfiévré, ce corps que la F

In [116]:
sent_extraction.preprocess(11)

Preprocessing of paragraph at line 11

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Ce que je recevais dans le bateau qui m’entraînait vers cet autre monde, c’était aussi la mémoire.
1: Le présent africain effaçait tout ce qui l’avait précédé.
2: La guerre, le confinement dans l’appartement de Nice (où nous vivions à cinq dans deux pièces mansardées, et même à six en comptant la bonne Maria dont ma grand-mère n’avait pas résolu de se passer), les rations, ou bien la fuite dans la montagne où ma mère devait se cacher, de peur d’être raflée par la Gestapo – tout cela s’effaçait, disparaissait, devenait irréel.
3: Désormais, pour moi, il y aurait avant et après l’Afrique.

On wolof_corpus: 
0: Ci biir saxaaru-géej gi sama xel a ngi wëndéelu, fu ne mu dem fa.
1: Ma dëkke fàttaliku, nag.
2: Fépp fa ma jotoon a jaar, lépp la ma masoon a dund laata may ñëw Afrig, daldi may songaat.
3: Loolu lan la woonati ?
4:  Xanaa liñ fi tudde woon Xare bu

In [117]:
sent_extraction.preprocess(12)

Preprocessing of paragraph at line 12

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: La liberté à Ogoja, c’était le règne du corps.
1: Illimité, le regard, du haut de la plate-forme de ciment sur laquelle était construite la maison, pareille à l’habitacle d’un radeau sur l’océan d’herbes.
2: Si je fais un effort de mémoire, je puis reconstituer les frontières vagues de ce domaine.
3: Quelqu’un qui aurait gardé la mémoire photographique du lieu serait étonné de ce qu’un enfant de huit ans pouvait y voir.
4: Sans doute un jardin.
5: Non pas un jardin d’agrément – existait-il dans ce pays quelque chose qui fût d’agrément ?
6:  Plutôt un espace d’utilité, où mon père avait planté des fruitiers, manguiers, goyaviers, papayers, et pour servir de haie devant la varangue, des orangers et des limettiers dont les fourmis avaient cousu la plupart des feuilles pour faire leurs nids aériens, débordant d’une sorte de duvet cotonneux qui abritait leur

In [119]:
sent_extraction.preprocess(13)

Preprocessing of paragraph at line 13

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Tout cela pourrait donner l’impression d’une vie coloniale, très organisée, presque citadine – ou du moins campagnarde à la façon de l’Angleterre ou de la Normandie d’avant l’ère industrielle.
1: Pourtant c’était la liberté totale du corps et de l’esprit.
2: Devant la maison, dans la direction opposée à l’hôpital où travaillait mon père, commençait une étendue sans horizon, avec une légère ondulation où le regard pouvait se perdre.
3: Au sud, la pente conduisait à la vallée brumeuse de l’Aiya, un affluent de la rivière Cross, et aux villages, Ogoja, Ijama, Bawop.
4: Vers le nord et l’est, je pouvais voir la grande plaine fauve semée de termitières géantes, coupée de ruisseaux et de marécages, et le début de la forêt, les bosquets de géants, irokos, okoumés, le tout recouvert par un ciel immense, une voûte de bleu cru où brûlait le soleil, et qu’envahiss

In [121]:
sent_extraction.preprocess(14)

Preprocessing of paragraph at line 14

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Ogoja me donnait une autre violence, ouverte, réelle, qui faisait vibrer mon corps.
1: C’était visible dans chaque détail de la vie et de la nature environnante.
2: Des orages tels que je n’en ai jamais vu ni rêvé depuis, le ciel d’encre zébré d’éclairs, le vent qui pliait les grands arbres autour du jardin, qui arrachait les palmes du toit, tourbillonnait dans la salle à manger en passant sous les portes et soufflait les lampes à pétrole.
3: Certains soirs, un vent rouge venu du nord, qui faisait briller les murs.
4: Une force électrique qu’il me fallait accepter, apprivoiser, et pour laquelle ma mère avait inventé un jeu, compter les secondes qui nous séparaient de l’impact de la foudre, l’entendre venir kilomètre après kilomètre, puis s’éloigner vers les montagnes.
5: Un après-midi, mon père opérait dans l’hôpital, quand la foudre est entrée par la p

In [122]:
sent_extraction.preprocess(15)

Preprocessing of paragraph at line 15

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: L’Afrique était puissante.
1: Pour l’enfant que j’étais, la violence était générale, indiscutable.
2: Elle donnait de l’enthousiasme.
3: Il est difficile d’en parler aujourd’hui, après tant de catastrophes et d’abandon.
4: Peu d’Européens ont connu ce sentiment.

On wolof_corpus: 
0: Duma ko tàyyee wax.
1: Kàttanu Afrig dafa jéggi woon dayo.
2: Doole joojoo nga woon fépp.
3: Ma nekk xale, muy yokk sama cawarte, di ma bégal.
4: Lu jafee waxtaane la tey ndax musiba yi ñu fi jot a jànkonteel ak it ni nit ñi bàyyeekoo, rawati-na fii ci Tugal.

-----------

The current sentences are:
french_corpus (index = 0) : L’Afrique était puissante.
wolof_corpus (index = 0) : Duma ko tàyyee wax.

You have stored the following sentences for the moment :

On french_corpus (length = 0): 


On wolof_corpus (length = 0): 


-----------

You have passed the following sentence

In [123]:
sent_extraction.preprocess(16)

Preprocessing of paragraph at line 16

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Le travail que faisait mon père au Cameroun d’abord, puis au Nigeria, créait une situation exceptionnelle.
1: La plupart des Anglais en poste dans la colonie exerçaient des fonctions administratives.
2: Ils étaient militaires, juges, district officers (ces D. O. dont les initiales, prononcées à l’anglaise, Di-O, m’avaient fait penser à un nom religieux, comme une variation sur le « Deo gratias » de la messe que ma mère célébrait sous la varangue chaque dimanche matin).
3: Mon père était l’unique médecin dans un rayon de soixante kilomètres.
4: Mais cette dimension que je donne n’a aucun sens : la première ville administrative était Abakaliki, à quatre heures de route, et pour y arriver il fallait traverser la rivière Aiya en bac, puis une épaisse forêt.
5: L’autre résidence d’un D.
6: O.
7: était à la frontière du Cameroun français, à Obudu, au pied des

D. O. must be transform to D.O. and remove the first parentheses.

In [134]:
sent_extraction.preprocess(17)

Preprocessing of paragraph at line 17

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Nous étions, mon frère et moi, les seuls enfants blancs de toute cette région.
1: Nous n’avons rien connu de ce qui a pu fabriquer l’identité un peu caricaturale des enfants élevés aux «colonies ».
2: Si je lis les romans «coloniaux» écrits par les Anglais de cette époque, ou de celle qui a précédé notre arrivée au Nigeria – Joyce Cary, par exemple, l’auteur de Missié Johnson –, je ne reconnais rien.
3: Si je lis William Boyd, qui a passé lui aussi une partie de son enfance dans l’Ouest africain britannique, je ne reconnais rien non plus : son père était D.
4: O.
5: (à Accra au Ghana, me semble-t-il).
6: Je ne sais rien de ce qu’il décrit, cette lourdeur coloniale, les ridicules de la société blanche en exil sur la côte, toutes les mesquineries auxquelles les enfants sont particulièrement attentifs, le dédain pour les indigènes, dont ils ne connaissent 

In [137]:
sent_extraction.preprocess(18)


Preprocessing of paragraph at line 18

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Nous n’allions pas à l’école.
1: Nous n’avions pas de club, pas d’activités sportives, pas de règle, pas d’amis au sens que l’on donne à ce mot en France ou en Angleterre.
2: Le souvenir que je garde de ce temps pourrait être celui passé à bord d’un bateau, entre deux mondes.
3: Si je regarde aujourd’hui la seule photo que j’ai conservée de la maison d’Ogoja (un cliché minuscule, le tirage 6 x 6 courant après la guerre), j’ai du mal à croire qu’il s’agit du même lieu : un grand jardin ouvert, où poussent en désordre des palmiers, des flamboyants, traversé par une allée rectiligne où est garée la monumentale Ford V 8 de mon père.
4: Une maison ordinaire, avec   un toit de tôle ondulée et, au fond, les premiers grands arbres de la forêt.
5: Il y a dans cette photo unique quelque chose de froid, presque austère, qui évoque l’empire, mélange de camp militai

In [138]:
sent_extraction.preprocess(19)


Preprocessing of paragraph at line 19

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: C’est ici, dans ce décor, que j’ai vécu les moments de ma vie sauvage, libre, presque dangereuse.
1: Une liberté de mouvement, de pensée et d’émotion que je n’ai plus jamais connue ensuite.
2: Les souvenirs trompent, sans doute.
3: Cette vie de liberté totale, je l’aurai sans doute rêvée plutôt que vécue.
4: Entre la tristesse du sud de la France pendant la guerre et la tristesse de la fin de mon enfance dans la Nice des années cinquante, rejeté de mes camarades de classe du fait de mon étrangeté, obsédé par l’autorité excessive de mon père, en butte à la très grande vulgarité des années lycée, des années scoutisme, puis pendant l’adolescence la menace d’avoir à partir faire la guerre pour maintenir les privilèges de la dernière société coloniale.

On wolof_corpus: 
0: Fii laa dunde dundug njaay-àll, moom sama bopp ba léeg-léeg sax mu mënoon maa gaañ.
1

In [141]:
sent_extraction.preprocess(20)


Preprocessing of paragraph at line 20

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Alors les jours d’Ogoja étaient devenus mon trésor, le passé lumineux que je ne pouvais pas perdre.
1: Je me souvenais de l’éclat sur la terre rouge, le soleil qui fissurait les routes, la course pieds nus à travers la savane jusqu’aux forteresses des termitières, la montée de l’orage le soir, les nuits bruyantes, criantes, notre chatte qui faisait l’amour avec les tigrillos sur le toit de tôle, la torpeur qui suivait la fièvre, à l’aube, dans le froid qui entrait sous le rideau de la moustiquaire.
2: Toute cette chaleur, cette brûlure, ce frisson.

On wolof_corpus: 
0: Tugal soof ma.
1: Ogosaa jommal ma, daanaka am ci man solo ba mu jéggi dayo.
2: May fàttaliku nag ray-rayu suuf su xonq si.
3: Tali yi jant bi xarat.
4: Ni nu daan rawantee tànki neen, di xuus ci àll bi, dem ba jub tatay max yi.
5: Te ngoon su ne, ngëlén li dellusi, guddi yi xumb be, fo 

In [142]:
sent_extraction.preprocess(21)


Preprocessing of paragraph at line 21

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Devant la maison d’Ogoja, passé la limite du jardin (plutôt un mur de broussailles qu’une haie taillée au cordeau), commençait la grande plaine d’herbes qui s’étendait jusqu’à la  rivière Aiya.
1: La mémoire d’un enfant exagère les distances et les hauteurs.
2: J’ai l’impression que cette plaine était aussi vaste qu’une mer.
3: Au bord du socle en ciment qui servait de trottoir à la case, je suis resté des heures, le regard perdu dans cette immensité, suivant les vagues du vent sur les herbes, m’arrêtant de loin en loin sur les petites trombes poussiéreuses qui dansaient au-dessus de la terre sèche, scrutant les taches d’ombre au pied des irokos.
4: J’étais vraiment sur le pont d’un bateau.
5: Le bateau, c’était la case, non seulement les murs de parpaing et le toit de tôle, mais tout ce qui portait la trace de l’empire britannique – à la manière du nav

In [144]:
sent_extraction.preprocess(22)


Preprocessing of paragraph at line 22

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: J’étais seulement un enfant, la puissance de l’empire m’indifférait assez.
1: Mais mon père en pratiquait la règle, comme si elle seule donnait un sens à sa vie.
2: Il croyait à la discipline, dans chaque geste de chaque jour : se lever tôt, faire aussitôt son lit, se laver à l’eau froide dans le bac de zinc, dont il fallait garder l’eau savonneuse pour mettre à tremper chaussettes et caleçons.
3: Les leçons avec ma mère chaque matin, orthographe, anglais, arithmétique.
4: La prière chaque soir, et le couvre-feu à neuf heures.
5: Rien de commun avec l’éducation à la française, les jeux du mouchoir et les trappe-trappe, les repas joyeux où tout le monde parle à la fois, et pour finir les douces chantefables que racontait ma grand-mère, les rêveries dans son lit en écoutant grincer la girouette, et la lecture des aventures d’une pie voyageuse au-dessus de

In [145]:
sent_extraction.preprocess(23)


Preprocessing of paragraph at line 23

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Je ne me souviens pas du jour où nous nous sommes aventurés, mon frère et moi, pour la première fois dans la savane.
1: Peut-être à l’instigation des enfants du village, la bande un peu hétéroclite qui comportait des tout-petits tout nus avec un gros ventre et des presque adolescents de douze, treize ans, vêtus comme nous d’un short kaki et d’une chemise, et qui nous avaient appris à ôter chaussures et chaussettes de laine pour courir pieds nus dans les herbes.
2: Ceux que je vois sur quelques photos de l’époque, autour de nous, très noirs, dégingandés, certainement moqueurs et combatifs, mais qui nous avaient acceptés malgré nos différences.

On wolof_corpus: 
0: Fàttalikuwuma bés bi ňu njëkkee sàcc dugg ci biir àll bi, maak sama mag ju góor.
1: Ñàkkul ne goney Ogosaa sooke woon mbir mi, nu topp ci.
2: Sunu gàngóor ga raxe lool, sax : ñii diy gone yu t

In [146]:
sent_extraction.preprocess(24)


Preprocessing of paragraph at line 24

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: C’était probablement interdit.
1: Mon père étant absent tout le jour, jusqu’à la nuit, nous avons dû comprendre que l’interdiction ne pouvait qu’être relative.
2: Ma mère était douce.
3: Sans doute était-elle occupée à d’autres choses, à lire ou à écrire, à l’intérieur de la maison pour échapper à la chaleur de l’après-midi.
4: Elle s’était faite africaine, à sa mesure, j’imagine qu’elle devait croire qu’il n’y avait pas d’endroit plus sûr au monde pour deux garçons de notre âge.

On wolof_corpus: 
0: Su nu Baay jàppoon keroog du baax ci nun.
1: Waaye ginnaaw da daan gudde ñibbisi ci kër gi, danu jàpp ni tere bi taxu koo am solo.
2: Sama yaay, moom, ku sedd la, noppal nu.
3: Ñàkkul sax mu ne ci leneen bés boobu, di jàng mbaa muy bind, ngir daw tàngaayu ngoon gi.
4: Dafa mujje gise boppam ni doomu Afrig, dem bay xalaat ni àll bee gën a wóor ci nun fépp f

In [147]:
sent_extraction.preprocess(25)


Preprocessing of paragraph at line 25

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Faisait-il chaud vraiment ?
1:  Je n’en ai aucun souvenir.
2: Je me souviens du froid de l’hiver, à Nice, ou à Roquebillière, je ressens encore l’air glacé qui soufflait dans les ruelles, un froid de glace et de neige, malgré nos guêtres et nos gilets en peau de mouton.
3: Mais je ne me rappelle pas avoir eu chaud à Ogoja.
4: Quand elle nous voyait sortir, ma mère nous obligeait à mettre nos casques Cawnpore – en réalité des chapeaux de paille qu’elle nous avait achetés avant notre départ dans un magasin de la vieille ville de Nice.
5: Mon père avait institué entre autres règles celle des chaussettes de laine et des chaussures de cuir cirées.
6: Dès qu’il partait pour son travail, nous nous mettions pieds nus pour courir.
7: Les premiers temps, je m’écorchais sur le ciment du sol en courant – je ne sais pourquoi, c’était toujours le gros orteil du pied 

In [148]:
sent_extraction.preprocess(26)


Preprocessing of paragraph at line 26

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Alors, un jour, nous avons couru tout seuls dans la plaine fauve, en direction de la rivière.
1: L’Aiya à cet endroit n’était pas très large, mais était animée d’un courant violent qui arrachait aux rives des mottes de boue rouge.
2: La plaine, de chaque côté de la rivière, paraissait sans bornes.
3: De loin en loin, au milieu de la savane, se dressaient de grands arbres au tronc très droit, dont j’ai su plus tard qu’ils servaient à fournir les planchers d’acajou des pays industriels.
4: Il y avait aussi des cotonniers, et des acacias épineux qui faisaient une ombre légère.
5: Nous courions presque sans nous arrêter, à perdre haleine, dans les hautes herbes qui fouettaient nos visages à hauteur des yeux, guidés par les fûts des grands arbres.
6: Aujourd’hui encore, quand je vois des images de l’Afrique, les grands parcs du Serengeti ou du Kenya, je ress

In [149]:
sent_extraction.preprocess(27)


Preprocessing of paragraph at line 27

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Au milieu de la plaine, à une distance suffisante pour que nous ne puissions plus voir notre case, il y avait des châteaux.
1: Le long d’une aire dénudée et sèche, des pans de murs rouge sombre, aux crêtes noircies par l’incendie, tels les remparts d’une ancienne citadelle.
2: De loin en loin, le long des murs, se dressaient des tours dont les sommets paraissaient becquetés d’oiseaux, déchiquetés, brûlés par la foudre.
3: Ces murailles occupaient une superficie aussi vaste qu’une ville.
4: Les murs, les tours étaient plus hauts que nous.
5: Nous n’étions que des enfants, mais dans mon souvenir j’imagine que ces murs devaient être plus hauts qu’un homme adulte, et certaines des tours devaient dépasser deux mètres.

On wolof_corpus: 
0: Biñ yeggee ci diggu joor gi, sori lool sunu kër, lañ gis i tabax yu mag a mag.
1: Ay miir yu xonq te lëndëm a toppoon ab

In [150]:
sent_extraction.preprocess(28)


Preprocessing of paragraph at line 28

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Nous savions que c’était la ville des termites.

On wolof_corpus: 
0: Nu daldi xoolante, xam ni yegsi nanu ci dëkku max yi.

-----------

Finished!


In [153]:
sent_extraction.preprocess(29)


Preprocessing of paragraph at line 29

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Comment l’avons-nous su ?
1:  Peut-être par mon père, ou bien par un des garçons du village.
2: Mais personne ne nous accompagnait.
3: Nous avons appris à démolir ces murs.
4: Nous avions dû commencer par jeter quelques pierres, pour sonder, pour écouter le bruit caverneux qu’elles faisaient en heurtant les termitières.
5: Puis nous avons frappé à coups de bâton les murs, les hautes tours, pour voir s’écrouler la terre poudreuse, mettre au jour les galeries, les bêtes aveugles qui y vivaient.
6: Le jour suivant, les ouvrières avaient colmaté les brèches, tenté de reconstruire les tours.
7: Nous frappions à nouveau, jusqu’à en avoir mal aux mains, comme si nous combattions un ennemi invisible.
8: Nous ne parlions pas, nous cognions, nous poussions des cris de rage, et de nouveaux pans de murs s’écroulaient.
9: C’était un jeu.
10: Était-ce un jeu ? Nous n

In [154]:
sent_extraction.preprocess(30)


Preprocessing of paragraph at line 30

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Les enfants du village n’étaient jamais avec nous quand nous partions détruire les termitières.
1: Sans doute cette rage de démolir les aurait-elle étonnés, eux qui vivaient dans un monde où les termites étaient une évidence, où ils jouaient un rôle dans les légendes.
2: Le dieu termite avait créé les fleuves au début du monde, et c’était lui qui gardait l’eau pour les habitants de la terre.
3: Pourquoi détruire sa maison ?
4:  La gratuité de cette violence pour eux n’aurait eu aucun sens : en dehors des jeux, bouger signifiait gagner de l’argent, recevoir une friandise, chasser quelque chose de vendable ou de comestible.
5: Les plus petits étaient sous la surveillance des plus grands, jamais seuls, jamais livrés à eux-mêmes.
6: Les jeux, les discussions et les menus travaux alternaient sans emploi du temps précis : ils ramassaient le bois mort et les b

In [156]:
sent_extraction.preprocess(31)


Preprocessing of paragraph at line 31

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Alors nous, nous étions sauvages comme de jeunes colons, sûrs de notre liberté, de notre impunité, sans responsabilités et sans aînés.
1: Quand mon père était absent, quand ma mère dormait, nous nous échappions, la prairie fauve nous happait.
2: Nous courions à toute vitesse, pieds nus, loin de la maison, à travers les hautes herbes qui nous aveuglaient, sautant par-dessus les rochers, sur la terre sèche et craquelée par la chaleur, jusqu’aux cités des termites.
3: Nous avions le cœur battant, la violence débordait avec notre souffle, nous prenions des pierres, des bâtons et nous frappions, frappions, nous faisions écrouler des pans de ces cathédrales, pour rien, simplement pour le bonheur de voir monter les nuages de poussière, entendre crouler les tours, résonner le bâton sur les murs durcis, pour voir s’offrir à la lumière les galeries rouges comme d

In [157]:
sent_extraction.preprocess(32)


Preprocessing of paragraph at line 32

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: J’ai pensé qu’il en aurait été autrement si nous étions restés à Ogoja, si nous étions devenus pareils aux Africains.
1: J’aurais appris à percevoir, à ressentir.
2: Comme les garçons du village, j’aurais appris à parler avec les êtres vivants, à voir ce qu’il y avait de divin dans les termites.
3: Je crois même qu’au bout d’un temps, je les aurais oubliés.

On wolof_corpus: 
0: Xéy-na mbir mi deme kon na neneen su ñu wéyoon di dëkk Ogosaa.
1: Dellu newaat doomu Afrig, yee kon na ma ci yu bari, ubbi itam sama xol.
2: Doon naa mën a wax ak rabi-àll yi, ñu may tontu.
3: Doon naa xam ni ay max duñuy mbindeefi neen.
4: Miin kon naa gis-gis boobu ba duma ko soxlaa fat ba koy fàtte.

-----------

The current sentences are:
french_corpus (index = 0) : J’ai pensé qu’il en aurait été autrement si nous étions restés à Ogoja, si nous étions devenus pareils aux Afr

In [159]:
sent_extraction.preprocess(33)


Preprocessing of paragraph at line 33

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Il y avait une hâte, une urgence.
1: Nous étions venus du bout du monde (car Nice était bien un autre bout du monde).
2: Nous étions venus d’un appartement au sixième étage d’un immeuble bourgeois, entouré d’un jardinet où les enfants n’avaient pas le droit de jouer, pour vivre en Afrique équatoriale, au bord d’une rivière boueuse, encerclés par la forêt.
3: Nous ne savions pas que nous allions en repartir.
4: Peut-être que nous avons pensé, comme tous les enfants, que nous allions y mourir.
5: Là-bas, de l’autre côté de la mer, le monde s’était figé dans le silence.
6: Une grand-mère avec ses contes, un grand-père avec sa voix chantante de Mauricien, des camarades de jeu, de classe, tout cela s’était glacé tels des jouets qu’on enferme dans une malle, telles les peurs qu’on laisse au fond des placards.
7: La plaine d’herbes avait aboli tout cela, dans 

In [160]:
sent_extraction.preprocess(34)


Preprocessing of paragraph at line 34

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Les fourmis étaient l’antiface de cette fureur.
1: Le contraire de la plaine d’herbes, de la violence destructrice.
2: Y avait-il des fourmis avant Ogoja ?
3:  Je ne m’en souviens pas.
4: Ou bien sans doute ces « fourmis d’Argentine », poussière noire qui envahissait chaque nuit la cuisine de ma grand-mère, reliant par des routes minuscules ses jardinières de rosiers en équilibre sur la gouttière et les amas de détritus qu’elle brûlait dans sa chaudière.

On wolof_corpus: 
0: Mellentaan yee doon giifal sunu mer, doon safaanub jooru ñax gi.
1: Ndax sax masoon naa gis ay mellentaan laata may dem dëkki Ogosaa ?
2:  Leerul ci sama bopp.
3: Su dee amoon na Tugal it, mellentaani Arsaŋtin la woon, baree-bare ba, fees sunnu waañ ba mujjee def pënd bu ñuul kukk, di lëkkale poti rósiyee yeek mbalit ma sama maam daan lakk ci biir jaarukaayam.

-----------

The cur

In [161]:
sent_extraction.preprocess(35)


Preprocessing of paragraph at line 35

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Les fourmis, à Ogoja, étaient des insectes monstrueux de la variété exectoïde, qui creusaient leurs nids à dix mètres de profondeur sous la pelouse du jardin, où devaient vivre des centaines de milliers d’individus.
1: Au contraire des termites, doux et sans défense, incapables dans leur cécité de causer le moindre mal, sauf celui de ronger le bois vermoulu des habitations et les troncs des arbres déchus, les fourmis étaient rouges, féroces, dotées d’yeux et de mandibules, capables de sécréter du poison et d’attaquer quiconque se trouvait sur leur chemin.
2: C’étaient elles les véritables maîtresses d’Ogoja.

On wolof_corpus: 
0: Mellentaani Ogosaa yee fi gënoon a raglu, daan gas seen néeg fa suuf gën a xóote.
1: Lu mat junniy-junni ci ñoom a fa daan dëkkandoo.
2: Ñu wute lool ak max yi.
3: Ñii ñoom, dañoo jàmbure ba dee.
4: Dañ daan ŋeeñ bant yeek dàtt

In [162]:
sent_extraction.preprocess(36)


Preprocessing of paragraph at line 36

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Je garde le souvenir cuisant de ma première rencontre avec les fourmis, dans les jours qui ont suivi mon arrivée.
1: Je suis dans le jardin, non loin de la maison.
2: Je n’ai pas remarqué le cratère qui signale l’entrée de la fourmilière.
3: Tout d’un coup, sans que je m’en sois rendu compte, je suis entouré par des milliers d’insectes.
4: D’où viennent-ils ?
5:  J’ai dû pénétrer dans la zone dénudée qui entoure l’orifice de leurs galeries.
6: Ce n’est pas tant des fourmis que je me souviens, que de la peur que je ressens.
7: Je reste immobile, incapable de fuir, incapable de penser, sur le sol tout à coup mouvant, formant un tapis de carapaces, de pattes et d’antennes qui tourne autour de moi et resserre son tourbillon, je vois les fourmis qui ont commencé à monter sur mes chaussures, qui s’enfoncent entre les mailles de ces fameuses chaussettes de lai

In [165]:
sent_extraction.preprocess(37)


Preprocessing of paragraph at line 37

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Une anecdote, une simple anecdote.
1: D’où vient que j’en garde la marque, comme si les morsures des fourmis guerrières étaient encore sensibles, que tout cela s’était passé hier ?
2:  Sans doute est-ce mêlé de légende, de rêve.
3: Avant ma naissance, raconte ma mère, elle voyage à cheval dans l’ouest du Cameroun, où mon père est médecin itinérant.
4: La nuit, ils campent dans des « cases de passage », de simples huttes de branches et de palmes au bord du chemin, où ils accrochent leurs hamacs.
5: Un soir, les porteurs sont venus les réveiller.
6: Ils portent des torches enflammées, ils parlent à voix basse, ils pressent mon père et ma mère de se lever.
7: Quand ma mère raconte cela, elle dit que ce qui l’a d’abord alarmée, c’est le silence, partout, alentour, dans la forêt, et les chuchotements des porteurs.
8: Dès qu’elle est debout, elle voit, à la l

In [167]:
sent_extraction.preprocess(38)


Preprocessing of paragraph at line 38

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Cette histoire, combien de fois ai-je entendu ma mère la raconter ?
1:  Au point de croire que cela m’était arrivé, de mêler le fleuve dévorant au tourbillon de fourmis qui m’avait assailli.
2: Le mouvement de giration des insectes autour de moi ne me quitte pas, et je reste figé dans un rêve, j’écoute le silence, un silence aigu, strident, plus effrayant qu’aucun bruit au monde.
3: Le silence des fourmis.

On wolof_corpus: 
0: Xew-xew boobu, ñaata yoon laa dégg sama yaay di ko nettali ?
1:  Du benn, du ñaar.
2: Dama mujjee dem ba yaakaar ne man mii maa dund loolu ci dëgg-dëgg, mu jaxasoo ci sama xel ak bés ba may mellentaan singalee ca Ogosaa.
3: Ñu ngi ma wër-ndomb, di dem ak a dikk.
4: Man ma taxaw ne yàcc, di xullee.
5: Waaye it maa ngi may nopp tekk-tekkaaral gi.
6: Dafa sew, leew, gën a raglu mbooleem riiri dunyaa yépp.
7: Tekkug mellentaan yi.

-

In [168]:
sent_extraction.preprocess(39)


Preprocessing of paragraph at line 39

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: À Ogoja, les insectes étaient partout.
1: Insectes de jour, insectes de nuit.
2: Ceux qui répugnent aux adultes n’ont pas le même effet sur les enfants.
3: Je n’ai pas besoin de faire de grands efforts d’imagination pour voir surgir à nouveau, chaque nuit, les armées de cafards – les cancrelats, comme les appelait mon grand-père, sujets d’une sirandane : kankarla, nabit napas kilot, il porte un habit, mais n’a pas de culotte.
4: Ils sortaient des fissures du sol, des lattes de bois du plafond, ils galopaient du côté de la cuisine.
5: Mon père les détestait.
6: Chaque nuit, il parcourait la maison, sa torche électrique d’une main, sa savate dans l’autre, pour une chasse vaine et sans fin.
7: Il était persuadé que les cafards étaient à l’origine de beaucoup de maladies, y compris du cancer.
8: Je me souviens de l’entendre dire : « Brossez bien les ongles 

In [169]:
sent_extraction.preprocess(40)


Preprocessing of paragraph at line 40

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Nous avions découvert d’autres compagnons de jeu : les scorpions.
1: Moins nombreux que les cafards, mais nous avions notre réserve.
2: Mon père, qui redoutait notre turbulence, avait installé sous la varangue, du côté le plus éloigné de sa chambre, deux trapèzes faits de bouts de corde et de vieux manches d’outils.
3: Nous utilisions les trapèzes pour un exercice particulier : suspendus par les jambes et la tête en bas, nous relevions délicatement la natte de paille que mon père avait mise pour amortir une éventuelle chute, et nous regardions les scorpions se figer dans une posture défensive, les pinces dressées et leur queue pointant son dard.
4: Les scorpions qui vivaient sous le tapis étaient généralement petits, noirs, probablement inoffensifs.
5: Mais de temps à autre, le matin, ils avaient été remplacés par un spécimen plus grand, de couleur blan

In [170]:
sent_extraction.preprocess(41)


Preprocessing of paragraph at line 41

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: J’ai du mal aujourd’hui à me souvenir des sentiments qui nous animaient.
1: Il me semble qu’il y avait dans ce rituel du trapèze et du scorpion quelque chose de respectueux, un respect évidemment inspiré par la crainte.
2: Comme les fourmis, les scorpions étaient les vrais habitants de ce lieu, nous ne pouvions être que des locataires indésirables et inévitables, destinés à nous en aller.
3: Des colons, en somme.

On wolof_corpus: 
0: Dina ma jafe tey ma wax lu nu doon tax a def loolu.
1: Xéy-na dan leen doon ormaal, worma juy biral ni nu leen ragale woon.
2: Mellentaan yeek jiit yee moomoon kër gi, nun nu fay ay doxandéem doŋŋ.
3: Xamuñu woon nuñ nuy def waaye it leeroon na leen ni, lu mu yàgg-yàgg, dinan leen mas a abali.
4: Ci gàttal, danu leen a jaay doole, ñëw ba ci seen biir kër sanc fa.

-----------

The current sentences are:
french_corpus (inde

In [176]:
sent_extraction.preprocess(42)


Preprocessing of paragraph at line 42

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Les scorpions furent un jour au centre d’une scène dramatique, dont le souvenir fait encore battre mon cœur aujourd’hui.
1: Mon père (ce devait être un dimanche matin, car il était à la maison) avait découvert dans un placard un scorpion de la variété blanche.
2: En fait, une femelle scorpion, qui transportait sa progéniture sur son dos.
3: Mon père aurait pu l’aplatir d’un coup de sa fameuse savate.
4: Il ne l’a pas fait.
5: Il est allé chercher dans sa pharmacie un flacon d’alcool à 90°, il en a aspergé le scorpion et a gratté une allumette.
6: Pour une raison que j’ignore, le feu a d’abord pris autour de l’animal, en formant un cercle de flammes bleues, et la femelle scorpion s’est arrêtée dans une posture tragique, les pinces levées au ciel, son corps bandé dressant au-dessus de ses enfants son crochet à venin au bout de sa glande, parfaitement visi

"audessus" must be transform to "au-dessus".

In [181]:
sent_extraction.preprocess(43)


Preprocessing of paragraph at line 43

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Je ne sais pourquoi, il me semble qu’à aucun autre endroit je n’ai ressenti cette impression de famille, de faire partie d’une cellule.
1: Après les journées brûlantes, à courir dans la savane, après l’orage et les éclairs, cette salle étouffante devenait pareille à la cabine d’un bateau fermée contre la nuit, tandis qu’au-dehors se déchaînait le monde des insectes.
2: Là j’étais vraiment à l’abri, comme à l’intérieur d’une grotte.
3: L’odeur de la soupe d’arachide, du foufou, du pain de manioc, la voix de mon père avec son accent chantant, en train de raconter les anecdotes de sa journée à l’hôpital, et le sentiment du danger au-dehors, l’armée invisible des papillons de nuit qui frappait les volets, les margouillats excités, la nuit chaude, tendue, non pas une nuit de repos et d’abandon comme autrefois, mais une nuit fiévreuse, harassante.
4: Et le go

In [183]:
sent_extraction.preprocess(44)


Preprocessing of paragraph at line 44

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Mon père est arrivé en Afrique en 1928, après deux années passées en Guyane anglaise comme médecin itinérant sur les fleuves.
1: Il en est reparti au début des années cinquante, lorsque l’armée a jugé qu’il avait dépassé l’âge de la retraite et qu’il ne pouvait plus servir.
2: Plus de vingt ans durant lesquels il a vécu en brousse (un mot qu’on disait alors, qu’on ne dit plus aujourd’hui), seul médecin sur des territoires grands comme des pays entiers, où il avait la charge de la santé de milliers de gens.

On wolof_corpus: 
0: Ci atum 1928 la Baay jëkk a teg tànk Afrig.
1: Fekk na booba mu def ñaari at ci Guwiyaan bi ci moomeelu Àngalteer.
2: Ndoorteelu atum 1950 la ko làrme bi yewwi, mu noppalu liggéey, jóge Afrig.
3: Kon, dund na lu ëpp ňaar-fukki at ci àll bi.
4: « Àll bi » : ñaari baat yu fi siiwoon te tey ñépp ragal leen a jëfandikoo.
5: Moom kenn

In [184]:
sent_extraction.preprocess(45)


Preprocessing of paragraph at line 45

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: L’homme que j’ai rencontré en 1948, l’année de mes huit ans, était usé, vieilli prématurément par le climat équatorial, devenu irritable à cause de la théophylline qu’il prenait pour lutter contre ses crises d’asthme, rendu amer par la solitude, d’avoir vécu toutes les années de guerre coupé du monde, sans nouvelles de sa famille, dans l’impossibilité de quitter son poste pour aller au secours de sa femme et de ses enfants, ou même de leur envoyer de l’argent.

On wolof_corpus: 
0: Atum 1948 lanu gisantewaat.
1: Awma woon lu dul juróom-ñetti at.
2: Teewul ma gis ci saa si ni mu sonnee, àddina soof ko, teel koo màggetloo.
3: Ginnaaw dafa amoon aasma, da daan jël téwófilin, garab googu tax koo gënatee naqadi deret.
4: Saa yu xelam demaan ci at yi ko Ñaareelu Xare Réewi Tubaab yi teqaleek njabootam, da doon gën a xund, mere ku ko deful dara.
5: Juróomi at 

In [185]:
sent_extraction.preprocess(46)


Preprocessing of paragraph at line 46

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: La plus grande preuve d’amour qu’il a donnée aux siens, c’est lorsqu’en pleine guerre, il traverse le désert jusqu’en Algérie, pour tenter de rejoindre sa femme et ses enfants et les ramener à l’abri en Afrique.
1: Il est arrêté avant d’atteindre Alger et il doit retourner au Nigeria.
2: Ce n’est qu’à la fin de la guerre qu’il pourra revoir sa femme et faire la connaissance de ses enfants au cours d’une brève visite dont je ne garde aucun souvenir.
3: De longues années d’éloignement et de silence, pendant lesquelles il a continué d’exercer son métier de médecin dans l’urgence, sans médicaments, sans matériel, tandis que partout dans le monde les gens s’entre-tuaient – cela devait être plus que difficile, cela devait être insoutenable, désespérant.
4: Il n’en a jamais parlé.
5: Il n’a jamais laissé entendre qu’il y ait eu dans son expérience quoi que ce 

In [187]:
sent_extraction.preprocess(47)


Preprocessing of paragraph at line 47

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Était-ce la guerre, cet interminable silence, qui avait fait de mon père cet homme pessimiste et ombrageux, autoritaire, que nous avons appris à craindre plutôt qu’à aimer ?
1: Était-ce l’Afrique ?
2: Alors, quelle Afrique ?
3:  Certainement pas celle qu’on perçoit aujourd’hui, dans la littérature ou dans le cinéma, bruyante, désordonnée, juvénile, familière, avec ses villages où règnent les matrones, les conteurs, où s’exprime à chaque instant la volonté admirable de survivre dans des conditions qui paraîtraient insurmontables aux habitants des régions plus favorisées.
4: Cette Afrique-là existait déjà avant la guerre, sans aucun doute.
5: J’imagine Douala, Port Harcourt, les rues encombrées de voitures, les marchés où courent les enfants luisant de sueur, les groupes de femmes parlant à l’ombre des arbres.
6: Les grandes villes, Onitsha et son marché 

In [188]:
sent_extraction.preprocess(48)


Preprocessing of paragraph at line 48

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Mon père m’a raconté un jour comment il avait décidé de partir au bout du monde, quand il a eu terminé ses études de médecine à l’hôpital Saint Joseph d’Elephant & Castle, à Londres.
1: Étant boursier du gouvernement, il devait effectuer un travail pour la communauté.
2: Il fut donc affecté au département des maladies tropicales à l’hôpital de Southampton.
3: Il prend le train, débarque à Southampton, s’installe dans une pension.
4: Son service ne débutant que trois jours plus tard, il flâne en ville, va voir les navires en partance.
5: À son retour à la pension, une lettre l’attend, un mot très sec du chef de l’hôpital disant : « Monsieur, je n’ai pas encore reçu votre carte de visite. » Mon père fait donc imprimer les fameuses cartes (j’en ai encore un exemplaire), juste son nom, sans adresse, sans titre.
6: Et il demande son affectation au ministère 

", pour son mariage puis pour la naissance de ses enfants," à supprimer.

In [189]:
sent_extraction.preprocess(49)


Preprocessing of paragraph at line 49

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: J’ai essayé d’imaginer ce qu’aurait pu être sa vie (donc la mienne) si, au lieu de fuir, il avait accepté l’autorité du chef de clinique de Southampton, s’était installé comme médecin de campagne dans la banlieue londonienne (ainsi que mon grand-père l’avait fait dans la banlieue parisienne), à Richmond, par exemple, ou même en Écosse (un pays qu’il avait toujours aimé).
1: Je ne veux pas parler des changements que cela aurait procurés à ses enfants (car naître ici ou là n’a pas dans le fond une importance considérable).
2: Mais ce que cela aurait changé en l’homme qu’il était, qui aurait mené une vie plus conforme, moins solitaire.
3: De soigner des enrhumés et des constipés, plutôt que des lépreux, des impaludés ou des victimes d’encéphalite léthargique.
4: D’apprendre à échanger, non sur le mode exceptionnel, par gestes, par interprète, ou dans cette

In [190]:
sent_extraction.preprocess(50)


Preprocessing of paragraph at line 50

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Il avait choisi autre chose.
1: Par orgueil sans doute, pour fuir la médiocrité de la société anglaise, par goût de l’aventure aussi.
2: Et cette autre chose n’était pas gratuite.
3: Cela vous plongeait dans un autre monde, vous emportait vers une autre vie.
4: Cela vous exilait au moment de la guerre, vous faisait perdre votre femme et vos enfants, vous rendait, d’une certaine façon, inéluctablement étranger.

On wolof_corpus: 
0: Deful loolu.
1: Moom kay, la gën a jafe la taamu.
2: Te dara dugalu ko ci woon lu dul yëg boppam ak ñeme lu metti.
3: Mu jaral ko lu ne nag, ba ci gàddaay dem fu sori, xare bi kalaate mu bàyyi njabootam Tugal, dëkk béréb bu mu xamul kenn, kenn xamu ko fa, saa yu rombee nit kooku naan ci xelam : « Moo, doxandéem bii, lan la fiy def ?  »

-----------

The current sentences are:
french_corpus (index = 0) : Il avait choisi autre 

In [4]:
sent_extraction.preprocess(51)


Preprocessing of paragraph at line 51

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: La première fois que j’ai vu mon père, à Ogoja, il m’a semblé qu’il portait des lorgnons.
1: D’où me vient cette idée ?
2:  Les lorgnons n’étaient déjà plus très courants à cette époque.
3: Peut-être qu’à Nice quelques vieux de la vieille avaient conservé cet accessoire, que j’imagine seyant parfaitement à d’anciens officiers russes de l’armée impériale, arborant moustache et favoris, ou bien inventeurs ruinés qui fréquentaient mes « tantes ».
4: Pourquoi lui ? En réalité, mon père devait porter des lunettes à la mode des années trente, fine monture d’acier et verres ronds qui reflétaient la lumière.
5: Les mêmes que je vois sur les portraits des hommes de sa génération, Louis Jouvet ou James Joyce (avec qui il avait du reste une certaine ressemblance).
6: Mais une simple paire de lunettes ne suffisait pas à l’image que j’ai gardée de cette première ren

In [5]:
sent_extraction.preprocess(52)


Preprocessing of paragraph at line 52

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Je crois que dans les premières heures qui ont suivi mon arrivée au Nigeria – la longue piste de Port Harcourt à Ogoja, sous la pluie battante, dans la Ford V8 gigantesque et futuriste, qui ne ressemblait à aucun véhicule connu – ce n’est pas l’Afrique qui m’a causé un choc, mais la découverte de ce père inconnu, étrange, possiblement dangereux.
1: En l’affublant de lorgnons, je justifiais mon sentiment.
2: Mon père, mon vrai père pouvait-il porter des lorgnons ?

On wolof_corpus: 
0: Yaakaar naa ne ba ma nee cëpp Niseryaa – nu jaar ci tali bu gudd ci diggante Poor Arkuur ak Ogosaa, toog ci sunu biir ponkalum Ford W8 bu kéemaane ba, taw bu mettiy dal – du Afrig moo ma teppati.
1: Baay ci boppam moo ma teppati.
2: Mii xel naan ma : « Moo, góor gu raglu gii, ku mu ?  » Ma jàpp ne mënul a doon sama baay, man mii.
3: Ngir gën a gëmloo loolu sama bopp, ma da

In [6]:
sent_extraction.preprocess(53)


Preprocessing of paragraph at line 53

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Son autorité a tout de suite posé un problème.
1: Nous avions vécu, mon frère et moi, dans une sorte de paradis anarchique à peu près dépourvu de discipline.
2: Le peu d’autorité auquel nous étions confrontés venait de ma grand-mère, une vieille dame généreuse et raffinée, qui était fondamentalement opposée à toute forme de châtiment corporel envers les enfants, lui préférant la raison et la douceur.
3: Mon grand-père maternel, lui, avait reçu dans sa jeunesse mauricienne des principes plus stricts, mais son grand âge, l’amour qu’il portait à ma grand-mère, et cette sorte de distance ennuyée propre aux gros fumeurs l’isolaient dans un réduit où il s’enfermait à clef pour, justement, y fumer en paix son caporal.

On wolof_corpus: 
0: Ci saa si la tàng-diineem ak bëgg yilifam jur fitna.
1: Maak sama mag kenn daawu nu wax déet.
2: Sunu maam ju jigéen daal,

In [7]:
sent_extraction.preprocess(54)


Preprocessing of paragraph at line 54

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Quant à ma mère, c’étaient la fantaisie et le charme.
1: Nous l’aimions, et j’imagine que nos bêtises la faisaient rire.
2: Je ne me rappelle pas l’avoir entendue élever la voix.
3: Dès lors, nous avions carte blanche pour faire régner dans le petit appartement une terreur enfantine.
4: Nous avons fait, dans les années qui ont précédé notre départ pour l’Afrique, des choses qui, avec le recul de l’âge, m’apparaissent en effet assez terribles : un jour, à l’instigation de mon frère, j’ai escaladé avec lui la rambarde du balcon (je la vois encore, nettement plus haute que moi) pour accéder à la gouttière, surplombant tout le quartier du haut des six étages.
5: Je pense que  mes grands-parents et ma mère ont été si effrayés que, lorsque nous avons consenti à revenir, ils ont oublié de nous punir.

On wolof_corpus: 
0: Sama yaay moom, deful woon dara muy da

In [11]:
sent_extraction.preprocess(55)


Preprocessing of paragraph at line 55

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Je me souviens aussi d’avoir été pris par des crises de rage, parce qu’on me refusait quelque chose, un bonbon, un joujou, bref pour une raison tellement insignifiante qu’elle ne m’a pas marqué, une rage telle que je jetais par la fenêtre tout ce qui me tombait sous la main, jusqu’à des meubles.
1: À ces moments, rien ni personne ne pouvait me calmer.
2: Parfois me revient encore la sensation de ces bouffées de colère, quelque chose que je ne peux comparer qu’à l’ivresse de l’éthéromane (l’éther qu’on faisait respirer aux enfants avant de leur arracher les amygdales).
3: La perte de contrôle, l’impression de flotter, et en même temps une extrême lucidité.
4: C’est l’époque où je suis également la proie de violents maux de tête, par instants si insupportables que je dois me cacher sous les meubles pour ne plus voir la lumière.
5: D’où venaient ces crises

In [12]:
sent_extraction.preprocess(56)


Preprocessing of paragraph at line 56

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Le voyage en Afrique met fin à tout cela.
1: Un changement radical : sur instructions de mon père, avant le départ, je dois me faire couper les cheveux, que j’ai portés jusque-là longs comme ceux d’un petit Breton, ce qui eut pour résultat de m’infliger un extraordinaire coup de soleil sur les oreilles, et de me faire rentrer dans le rang de la normalité masculine.
2: Plus jamais je ne ressentirai ces atroces migraines, plus jamais je ne pourrai donner libre cours aux crises de colère de ma petite enfance.
3: L’arrivée en Afrique a été pour moi l’entrée dans l’antichambre du monde adulte.

On wolof_corpus: 
0: Gàddaay jëm Afrig a dakkal loolu lépp.
1: Ni Yàllaak Yaali : ci ndigalu Baay, ñu xuuf samay karaw yu gudd yu goneg Bërëtaañ ya, naaj wiy jafal samay nopp.
2: Ma daldi dellu nekk xale bu góor.
3: Ca la samay metiti bopp ya dakke ba fàww, ma bàyyi t

In [13]:
sent_extraction.preprocess(57)


Preprocessing of paragraph at line 57

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: À l’âge de trente ans, mon père quitte Southampton à bord d’un cargo mixte à destination de Georgetown, en Guyane britannique.
1: Les rares photos de lui à cette époque montrent un homme robuste, à l’allure sportive, vêtu de façon élégante, complet veston, chemise à col dur, cravate, gilet, souliers de cuir noir.
2: Bientôt huit ans qu’il est parti de Maurice, après l’expulsion de sa famille de la maison natale, le fatal jour de l’an de 1919.
3: Dans le petit carnet où il a consigné les événements marquants des derniers jours passés à Moka, il écrit : « À présent, je n’ai plus qu’un désir, partir très loin d’ici et ne jamais revenir. » La Guyane, c’était effectivement l’autre extrémité du monde, les antipodes de Maurice.

On wolof_corpus: 
0: Ci fànweeri atam la Baay jël gaal, jóge Saawusamton jëm Sorstaawun, ca Guwiyaan ba newoon ci loxoy Àngale yi.
1:

In [6]:
sent_extraction.preprocess(58)


Preprocessing of paragraph at line 58

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Est-ce le drame de Moka qui a justifié cet éloignement ?
1:  Il y a eu sans doute au moment de son départ une détermination qui ne l’a jamais quitté.
2: Il ne pouvait pas être comme les autres.
3: Il ne pouvait pas oublier.
4: Il ne parlait jamais de l’événement qui avait été à l’origine de la dispersion de tous les membres de sa famille.
5: Sauf, de temps en temps, pour laisser échapper un éclat de colère.

On wolof_corpus: 
0: Ndax musibam Mokaa waraloon mu dëddu ?
1:  Pas-pas ba mu wone ba muy laxas, du woon neen, du woon caaxaan.
2: Ñépp mënoon nañoo fàtte, ba mu des moom.
3: Manul woon a mel ni ňeneen ňi.
4: Manul woon a fàtte.
5: Waxam nag, bariwul woon ci mbir moomu.
6: Xanaa léeg-léeg rekk mu mer, xultu, ay kàddu rëcc ko.
7: Wànte daal mbóot moomu, maanaam la tasoon seen kër, daawu ko sulli mukk.

-----------

The current sentences are:
french_c

In [7]:
sent_extraction.preprocess(59)


Preprocessing of paragraph at line 59

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Pendant sept ans il étudie à Londres, d’abord dans une école d’ingénieur, puis à la faculté de médecine.
1: Sa famille est ruinée, et il ne peut compter que sur la bourse du gouvernement.
2: Il ne peut pas se permettre d’échouer.
3: Il fait une spécialité de médecine tropicale.
4: Il sait déjà qu’il n’aura pas les moyens de s’installer comme médecin privé.
5: L’épisode de la carte de visite exigée par le médecin-chef de l’hôpital de Southampton ne sera que le prétexte à rompre avec la société européenne.

On wolof_corpus: 
0: Lu tollook juróom-ňaari at, mu nekk Londar di jàng.
1: Eñseñeer la jëkkon a bëgg nekk waaye dafa ca mujjee génn, daldi taamu njàngum faj.
2: Jafe-jafe yiy way-juram doon jànkoonteel tax ba mu war a sukkandiku ci ndimbalu gornmaa bi.
3: Lajj moom, xel daju ko.
4: Fàww xelam gën a màcc ci pajum feebari réewi Afrig yi.
5: Xamoon na ni

In [8]:
sent_extraction.preprocess(60)


Preprocessing of paragraph at line 60

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: La seule part de douceur dans sa vie, à ce moment-là, c’est la fréquentation de son oncle à Paris, la passion qu’il éprouve pour sa cousine germaine, ma mère.
1: Les congés qu’il passe en France auprès d’eux sont le retour imaginaire vers un passé qui n’est plus.
2: Mon père est né dans la même maison que son oncle, à tour de rôle ils y ont grandi, ils ont connu les mêmes lieux, les mêmes secrets, les mêmes cachettes, ils se sont baignés dans le même ruisseau.
3: Ma mère n’a pas vécu là-bas (elle est née à Milly), mais elle en a toujours entendu parler par son père, cela fait partie de son passé, pour elle cela a le goût d’un rêve inaccessible et familier (car, en ce temps-là, Maurice est si loin qu’on ne peut qu’en rêver).
4: Mon père et elle sont unis par ce rêve, ils sont ensemble comme les exilés d’un pays inaccessible.

On wolof_corpus: 
0: Lenn do

In [9]:
sent_extraction.preprocess(61)


Preprocessing of paragraph at line 61

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Il n’importe.
1: Mon père a décidé de partir, il partira.
2: Le Colonial Office vient de lui attribuer un poste de médecin sur les fleuves de Guyane.
3: Dès qu’il arrive, il affrète une pirogue munie d’un toit de palmes et propulsée par un moteur Ford à axe long.
4: À bord de sa pirogue, accompagné par l’équipe, infirmiers, pilote, guide et interprète, il remonte les rivières : le Mazaruni, l’Essequeibo, le Kupurung, le Demerara.

On wolof_corpus: 
0: Loolu lépp amul solo ci gis-gisu Baay.
1: Jàpp na ni fàww mu dem.
2: Nguurug Àngalteer daldi ko yabal ca dexi Guwiyaan ya.
3: Dafa fa yegg rekk, luye gaal ànd ak fajkat ya mu yilif, dawalkati gaal ya ak i làpptoo, ñuy xuus ca dex ga, di rombiy dëkk di dem : Masurini, Isekeyboo, Kupuruŋ, Demeraraa.

-----------

The current sentences are:
french_corpus (index = 0) : Il n’importe.
wolof_corpus (index = 0) : 

In [10]:
sent_extraction.preprocess(62)


Preprocessing of paragraph at line 62

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Il prend des photos.
1: Avec son Leica à soufflet, il collectionne des clichés en noir et blanc qui représentent mieux que des mots son éloignement, son enthousiasme devant la beauté de ce nouveau monde.
2: La nature tropicale n’est pas une découverte pour lui.
3: À Maurice, dans les ravins, sous le pont de Moka, la rivière Terre-Rouge n’est pas différente de ce qu’il trouve en haut des fleuves.
4: Mais ce pays est immense, il n’appartient pas encore tout à fait aux hommes.
5: Sur ses photos paraissent la solitude, l’abandon, l’impression d’avoir touché à la rive la plus lointaine du monde.
6: Du débarcadère du Berbice, il photographie la nappe bistre sur laquelle glisse une pirogue, contre un village de tôle semé d’arbres malingres.
7: Sa maison, une sorte de chalet de planches sur pilotis, au bord d’une route vide, flanquée d’un seul palmier absurde.


In [11]:
sent_extraction.preprocess(63)


Preprocessing of paragraph at line 63

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Les photos que mon père a aimé prendre, ce sont celles qui montrent l’intérieur du continent, la force inouïe des rapides que sa pirogue doit remonter, halée sur des rondins, à côté des marches de pierre où l’eau cascade, avec sur chaque rive les murs sombres de la forêt.

On wolof_corpus: 
0: Li ëpp ci nataal yi Baay jël, dañuy wone xolub jéeri ji ak tamit doole ginnax yi, doole ju kenn àttanul.
1: Nga gis ni leen gaal giy songe, ak ni ndox miy balle ci eskale xeer yi, àll bi wër ko, lëndëm këruus.

-----------

The current sentences are:
french_corpus (index = 0) : Les photos que mon père a aimé prendre, ce sont celles qui montrent l’intérieur du continent, la force inouïe des rapides que sa pirogue doit remonter, halée sur des rondins, à côté des marches de pierre où l’eau cascade, avec sur chaque rive les murs sombres de la forêt.
wolof_corpus (inde

In [12]:
sent_extraction.preprocess(64)


Preprocessing of paragraph at line 64

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Les chutes de Kaburi, sur le Mazaruni, l’hôpital de Kamakusa, les maisons de bois le long du fleuve, les boutiques de chercheurs de diamants.
1: Soudain une bonace sur un bras du Mazaruni, un miroir d’eau qui étincelle et entraîne vers la rêverie.
2: Sur la photo apparaît l’étrave de la pirogue en train de descendre le fleuve, je la regarde et je sens le vent, l’odeur de l’eau, j’entends malgré le grondement du moteur le crissement incessant des insectes dans la forêt, je perçois l’inquiétude qui naît à l’approche de la nuit.
3: À l’embouchure du rio Demerara, les palans chargent le sucre demerara à bord des cargos rouillés.
4: Et sur une plage, où viennent mourir les vagues du sillage, deux enfants indiens me regardent, un petit garçon de six ans environ et sa sœur à peine plus âgée, tous deux ont le ventre distendu par la parasitose, leurs cheveux trè

In [13]:
sent_extraction.preprocess(65)



Preprocessing of paragraph at line 65

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Plus tard, longtemps après, je suis allé à mon tour au pays des Indiens, sur les fleuves.
1: J’ai connu des enfants semblables.
2: Sans doute le monde a-t-il changé beaucoup, les rivières et les forêts sont moins pures qu’elles n’étaient au temps de la jeunesse de mon père.
3: Pourtant il m’a semblé comprendre le sentiment d’aventure qu’il avait éprouvé en débarquant au port de Georgetown.
4: Moi aussi, j’ai acheté une pirogue, j’ai voyagé debout à la proue, les orteils écartés pour mieux agripper le bord, balançant la longue perche dans mes mains, regardant les cormorans s’envoler devant moi, écoutant le vent souffler dans mes oreilles et les échos du moteur de hors-bord s’enfoncer derrière moi dans l’épaisseur de la forêt.
5: En examinant la photo prise par mon père à l’avant de la pirogue, j’ai reconnu la proue au museau un peu carré, la corde d’amar

In [15]:
sent_extraction.preprocess(66)


Preprocessing of paragraph at line 66

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Quand je suis revenu des terres indiennes, mon père était déjà malade, enfermé dans son silence obstiné.
1: Je me souviens de l’étincelle dans ses yeux quand je lui ai raconté que j’avais parlé de lui aux Indiens, et qu’ils l’invitaient à retourner sur les fleuves, qu’en échange de son savoir et de ses médicaments, ils lui offraient une maison et la nourriture pour le temps qu’il voudrait.
2: Il a eu un léger sourire, il a dit, je crois : « Il y a dix ans, j’y serais allé. » C’était trop tard, le temps ne se remonte pas, même dans les rêves.

On wolof_corpus: 
0: Bi ma jógee ci dëkki Enjeŋ yi, fekkoon na Baay tawat, tëju daanaka ci biir boppam, ni du wax te soo waxeek moom du la tontu.
1: Waaye maa ngi fàttaliku ferňeent ba takk ciy bëtam bés ba ma ni ko Enjeŋ yi namm nañ ko lool te su doon seen sago mu dellu fa, nekkaat seen doktoor, ñu ameel ko léppam

In [16]:
sent_extraction.preprocess(67)


Preprocessing of paragraph at line 67

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: C’est la Guyane qui a préparé mon père à l’Afrique.
1: Après tout ce temps passé sur les fleuves, il ne pouvait pas revenir en Europe – encore moins à Maurice, ce petit pays où il se sentait à l’étroit au milieu de gens égoïstes et vaniteux.
2: Un poste venait d’être créé en Afrique de l’Ouest, dans la bande de terre reprise à l’Allemagne à la fin de la Première Guerre mondiale, et qui comprenait l’est du Nigeria et l’ouest du Cameroun, sous mandat britannique.
3: Mon père s’est porté volontaire.
4: Début 1928, il est dans un bateau qui longe la côte de l’Afrique à destination de Victoria, sur la baie du Biafra.

On wolof_corpus: 
0: Guwiyaan la Baay waajale Afrig.
1: Xel dajul woon mu dellusi Tugal astamaak Móris, dun bu xat boobu te nit ña siis te fuuy.
2: Àngale ya nangu woon ci Almaañ penkub Niseryaa ak sowub Kamerun daldi ko woo, mu war faa nekki f

In [17]:
sent_extraction.preprocess(68)


Preprocessing of paragraph at line 68

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: C’est ce même voyage que j’ai fait, vingt ans plus tard, avec ma mère et mon frère, pour retrouver mon père au Nigeria après la guerre.
1: Mais lui n’est pas un enfant qui se laisse porter par le courant des événements.
2: Il a alors trentedeux ans, c’est un homme endurci par deux années d’expérience médicale en Amérique tropicale, il connaît la maladie et la mort, il les a côtoyées chaque jour, dans l’urgence, sans protection.

On wolof_corpus: 
0: Xare bi jeex, ñu tekk ci ñaar-fukki at, ma ànd ak sama yaay ak sama mag ju góor, nun ñett ñépp nu jaar fa mu jaaroon, dem fekki ko Afrig.
1: Waaye mbir yi xawoon nañoo woroo : bi muy def tukki boobu, mu ngi tollu ci fanweeri at ak ňaar, di góor gu dëgër, làmboo ňaari ati paj ca réewu Enjeŋ ya, miin fa feebar ak dee ak ňàkkum kaaraange.

-----------

The current sentences are:
french_corpus (index = 0) : C’es

In [18]:
sent_extraction.preprocess(69)


Preprocessing of paragraph at line 69

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Son frère Eugène, qui a été médecin avant lui en Afrique, le lui a certainement dit : il ne va pas dans un pays facile.
1: Le Nigeria est sans doute « pacifié », occupé par l’armée britannique.
2: Mais c’est une région où la guerre est permanente, guerre des hommes entre eux, guerre de la pauvreté, guerre des mauvais traitements et de la corruption hérités de la colonisation, guerre microbienne surtout.
3: Au Calabar, au Cameroun, l’ennemi n’est plus Aro Chuku et son oracle, ni les armées des Foulanis et leurs longues carabines venues d’Arabie.
4: Les ennemis s’appellent kwashiorkor, bacille virgule, ténia, bilharzia, variole, dysenterie amibienne.
5: Face à ces ennemis, la trousse de médecin de mon père doit lui paraître bien légère.
6: Scalpel, pinces à clampser, trépan, stéthoscope, garrots, et quelques outils de base, dont la seringue de laiton avec

In [19]:
sent_extraction.preprocess(70)


Preprocessing of paragraph at line 70

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Dans l’Ouest africain, il va rester vingt-deux ans, jusqu’à la limite de ses forces.
1: Ici, il connaîtra tout, depuis l’enthousiasme du commencement, la découverte des grands fleuves, le Niger, le Bénoué, jusqu’aux hautes terres du Cameroun.
2: Il partagera l’amour et l’aventure avec sa femme, à cheval sur les sentiers de montagne.
3: Puis la solitude et l’angoisse de la guerre, jusqu’à l’usure, jusqu’à l’amertume des derniers instants, ce sentiment d’avoir dépassé la mesure d’une vie.

On wolof_corpus: 
0: Afrig, dafa fa dem ba jeex.
1: Toog na fa ñaar-fukki at ak ñaar, lépp mu xame ko fa.
2: Fa la jëkkee xam lu mbégte tekki, fa la jëkkee gis dex yu mag a mag ya, muy Niseer di Bénuwe.
3: Tangori Kamerun yi tamit, jamono jooju la leen njëkk a teg bët.
4: Te loolu lépp am lu ko ëpp solo, te muy lépp li ñu ànd dund, mook soxnaan.
5: Mu di ko won saa su n

In [20]:
sent_extraction.preprocess(71)


Preprocessing of paragraph at line 71

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Tout cela, je ne l’ai compris que beaucoup plus tard, en partant comme lui, pour voyager dans un autre monde.
1: Je l’ai lu, non pas sur les rares objets, masques, statuettes, et les quelques meubles qu’il avait rapportés du pays ibo et des Grass Fields du Cameroun.
2: Ni même en regardant les photos qu’il a prises pendant les premières années, à son arrivée en Afrique.
3: Je l’ai su en redécouvrant, en apprenant à mieux lire les objets de la vie quotidienne qui ne l’avaient jamais quitté, même pendant sa retraite en France : ces tasses, assiettes de métal émaillé bleu et blanc faites en Suède, ces couverts en aluminium avec lesquels il avait mangé pendant toutes ces années, ces gamelles emboîtées qui lui servaient en campagne, dans les cases de passage.
4: Et tous les autres objets, marqués, cabossés par les cahots, portant la trace des pluies diluvien

In [23]:
sent_extraction.preprocess(72)


Preprocessing of paragraph at line 72

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: C’était comme s’il n’avait jamais quitté l’Afrique.
1: À son retour en France, il avait gardé les habitudes de son métier, levé à six heures, habillé (toujours de son pantalon de toile kaki), ses chaussures cirées, son chapeau sur la tête, pour aller faire ses courses au marché – comme jadis il partait pour la tournée des lits à l’hôpital –, de retour chez lui à huit heures, pour préparer le repas – avec la minutie d’une intervention chirurgicale.
2: Il avait conservé toutes les manies des anciens militaires.
3: L’homme qui avait reçu l’entraînement des médecins en pays lointains – être ambidextre, capable de s’opérer soi-même en se servant d’un miroir ou de recoudre sa hernie.
4: L’homme aux mains calleuses de chirurgien, qui pouvait scier un os ou placer une attelle, qui savait faire des nœuds et des épissures – cet homme n’utilisait plus son énergie 

In [25]:
sent_extraction.preprocess(73)


Preprocessing of paragraph at line 73

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: L’Afrique, pour mon père, a commencé en touchant la Gold Coast, à Accra.
1: Image caractéristique de la Colonie : des voyageurs européens, vêtus de blanc et coiffés du casque Cawnpore, sont débarqués dans une nacelle et transportés jusqu’à terre à bord d’une pirogue montée par des Noirs.
2: Cette Afrique-là n’est pas très dépaysante : c’est l’étroite bande qui suit le contour de la côte, depuis la pointe du Sénégal jusqu’au golfe de Guinée, que connaissent tous ceux qui viennent des métropoles pour faire des affaires et s’enrichir promptement.
3: Une société qui, en moins d’un demi-siècle, s’est architecturée en castes, lieux réservés, interdits, privilèges, abus et profits.
4: Banquiers, agents commerciaux, administrateurs civils ou militaires, juges, policiers et gendarmes.
5: Autour d’eux, dans les grandes villes portuaires, Lomé, Cotonou, Lagos, com

In [26]:
sent_extraction.preprocess(74)


Preprocessing of paragraph at line 74

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: C’est cette image que mon père a détestée.
1: Lui qui avait rompu avec Maurice et son passé colonial, et se moquait des planteurs et de leurs airs de grandeur, lui qui avait fui le conformisme de la société anglaise, pour laquelle un homme ne valait que par sa carte de visite, lui qui avait parcouru les fleuves sauvages de Guyane, qui avait pansé, recousu, soigné les chercheurs de diamants et les Indiens sous-alimentés ; cet homme ne pouvait pas ne pas vomir le monde colonial et son injustice outrecuidante, ses cocktails parties et ses golfeurs en tenue, sa domesticité, ses maîtresses d’ébène prostituées de quinze ans introduites par la porte de service, et ses épouses officielles pouffant de chaleur et faisant rejaillir leur rancœur sur leurs serviteurs pour une question de gants, de poussière ou de vaisselle cassée.

On wolof_corpus: 
0: Njaayum puuka

In [27]:
sent_extraction.preprocess(75)


Preprocessing of paragraph at line 75

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: En parlait-il ?
1: D’où me vient cette instinctive répulsion que j’ai ressentie depuis l’enfance pour le système de la Colonie ?
2:  Sans doute ai-je capté un mot, une réflexion, à propos des ridicules des administrateurs, tel le district officer d’Abakaliki que mon père m’emmenait voir parfois et qui vivait au milieu de sa meute de pékinois nourris au filet de bœuf et aux petits gâteaux, abreuvés uniquement à l’eau minérale.
3: Ou bien les récits de grands Blancs qui voyageaient en convoi, à la chasse aux lions et aux éléphants, armés de fusils à lunette et de balles explosives, et  qui, lorsqu’ils croisaient mon père dans ces contrées perdues, le prenaient pour un organisateur de safaris et l’interrogeaient sur la présence d’animaux sauvages, à quoi mon  père répondait : « Depuis vingt ans que je suis ici, je n’en ai jamais vu un, à moins que vous ne 

In [28]:
sent_extraction.preprocess(76)


Preprocessing of paragraph at line 76

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Peut-être est-ce le regard de ma mère sur ce continent à la fois si neuf et si malmené par le monde moderne ?
1:  Je ne me souviens pas de ce qu’elle nous disait, à mon frère et à moi, quand elle nous parlait du pays où elle avait vécu avec mon père, où nous devions le rejoindre un jour.
2: Je sais seulement que, lorsque ma mère a décidé de se marier avec mon père, et d’aller vivre au Cameroun, ses amies parisiennes lui ont dit : « Quoi, chez les sauvages ?» et qu’elle, après tout ce que mon père lui avait raconté, n’a pu que répondre : « Ils ne sont pas plus sauvages que les gens à Paris ! »

On wolof_corpus: 
0: Amaana sunu gis-gisu yaay ci Afrig mujje noo wàll.
1: Afrig ma doon door a juddu, daanaka, te ñu teel koo teg coono àddina yu dul jeex.
2: Dëgg la, fàttalikuwuma la mu nu ci daan wax.
3: Li am daal, moo di ne ba mu nee dafa fay fekki Baay, xar

In [29]:
sent_extraction.preprocess(77)


Preprocessing of paragraph at line 77

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Après Lagos, Owerri, Abo non loin du fleuve Niger.
1: Déjà mon père est loin de la zone « civilisée ».
2: Il est devant les paysages de l’Afrique équatoriale tels que les décrit André Gide dans son Voyage au Congo (à peu près contemporain de l’arrivée de mon père au Nigeria) : l’étendue du fleuve, vaste comme un bras de mer, sur lequel naviguent pirogues et bateaux à aubes, et les affluents, la rivière d’Ahoada avec ses « sampans » aux toits de palmes, poussés par des perches, et plus près de la côte, la rivière Calabar, et l’échancrure du village d’Obukun, taillée à coups de machette dans l’épaisseur de la forêt.
3: Ce sont les premières images que mon père reçoit du pays où il va passer la plus grande partie de sa vie active, du pays qui va devenir, par force et par nécessité, son vrai pays.

On wolof_corpus: 
0: Romb Legoos ak Ówéri, daldi agsi Aboo,

In [30]:
sent_extraction.preprocess(78)


Preprocessing of paragraph at line 78

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: J’imagine son exaltation à l’arrivée à Victoria, après vingt jours de voyage.
1: Dans la collection de clichés pris par mon père en Afrique, il y a une photo qui m’émeut particulièrement, parce que c’est celle qu’il a choisi d’agrandir pour en faire un tableau.
2: Elle traduit son impression d’alors, d’être au commencement, au seuil de l’Afrique, dans un endroit presque vierge.
3: Elle montre l’embouchure de la rivière, à l’endroit où l’eau douce se mêle à la mer.
4: La baie de Victoria dessine une courbe terminée par une pointe de terre où les palmiers sont inclinés dans le vent du large.
5: La mer déferle sur les roches noires et vient mourir sur la plage.
6: Les embruns apportés par le vent recouvrent les arbres de la forêt, se mêlent à la vapeur des marécages et de la rivière.
7: Il y a du mystère et de la sauvagerie, malgré la plage, malgré les pal

In [31]:
sent_extraction.preprocess(79)


Preprocessing of paragraph at line 79

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Pour cela, je sens son impatience, son grand désir de pénétrer à l’intérieur du pays, pour commencer son métier de médecin.
1: De Victoria, les pistes le conduisent à travers le mont Cameroun vers les hauts plateaux où il doit prendre son poste, à Bamenda.
2: C’est là qu’il va travailler pendant les premières années, dans un hôpital à moitié en ruine, un dispensaire de bonnes sœurs irlandaises, murs de boue séchée  et toit de palmes.
3: C’est là qu’il va passer les années les plus heureuses de sa vie.

On wolof_corpus: 
0: Looloo waral may yëg ni mu yàkkamtee dugg ci biir réew mi, ngir gaaw a tàmbalee faj nit ñi.
1: Yooni Wiktoryaa yaa nga jëm tundu Kamerun, ca dëkki tangori Bamendaa yu kowe ya mu waroon a doore liggéeyam.
2: Dispañseeru waa kër làbbe yu jigéen a ko fa doon xaar.
3: Dispañseer bu màggat ak i miiri banam yuñ teg xobi garabu-tiir.
4: Mbég

In [32]:
sent_extraction.preprocess(80)


Preprocessing of paragraph at line 80

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: Sa maison, c’est Forestry House, une vraie maison en bois à étage, recouverte d’un toit de feuilles que mon père va s’employer à reconstruire avec le plus grand soin.
1: En contrebas, dans la vallée, non loin des prisons, se trouve la ville haoussa avec ses remparts de pisé et ses hautes portes, telle qu’elle était au temps de la gloire de l’Adamawa.
2: Un peu à l’écart, l’autre ville africaine, le marché, le palais du roi de Bamenda, et la maison de passage du district officer et des officiers de Sa Majesté (ils ne sont venus qu’une seule fois, pour décorer le roi).
3: Une photo prise par mon père, sans doute un peu satirique, montre ces messieurs du gouvernement britannique, raides dans leurs shorts et leurs chemises empesées, coiffés du casque, mollets moulés dans leurs bas de laine, en train de regarder le défilé des guerriers du roi, en pagne et la

In [33]:
sent_extraction.preprocess(81)


Preprocessing of paragraph at line 81

-----------
-----------

Do you want to process the following sentences:

On french_corpus: 
0: C’est à Bamenda que mon père emmène ma mère après leur mariage, et Forestry House est leur première maison.
1: Ils installent leurs meubles, les seuls meubles qu’ils ont jamais achetés et qu’ils emporteront avec eux partout : des tables, des fauteuils taillés dans des troncs d’iroko, décorés de sculptures traditionnelles des hauts plateaux de l’Ouest camerounais, léopards, singes, antilopes.
2: La photo que mon père prend de leur salon à Forestry House montre un décor très « colonial » : au-dessus du manteau de la cheminée (il fait froid à Bamenda en hiver) est accroché un grand bouclier en peau d’hippopotame, assorti de deux lances croisées.
3: Il s’agit vraisemblablement d’objets laissés là par un précédent occupant, car cela ne ressemble pas à ce que mon père pouvait rechercher.
4: Les meubles sculptés, en revanche, l’ont accompagné jusqu’en France.
